In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
OUTPUT_FOLDER = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/grayscale_feature/labelled_images"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
# Loading stored reference features from CSV
def load_reference_features(csv_file):
    return pd.read_csv(csv_file, header=None).values.flatten()

In [ ]:
# Extracting features from an image
def extract_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image {image_path}")
        return None

    img = cv2.resize(img, (128, 128))  # Resize for consistency
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # GLCM Texture Features
    from skimage.feature import graycomatrix, graycoprops # Import graycomatrix and graycoprops
    glcm = graycomatrix(gray, distances=[5], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

    # Compute histogram (color features)
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]).flatten()

    # Edge detection (Canny)
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges) / (128 * 128)

    # Combine features (texture + color + edge) to match the original feature extraction
    return np.hstack([contrast, correlation, energy, homogeneity, hist, edge_density])

In [ ]:
# Classifing and rename images
def classify_and_rename_images(image_folder, feature_csv, output_folder, threshold=0.8):
    # Loading stored features from CSV
    reference_features = load_reference_features(feature_csv)

    # Checking output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterating over all images in the folder
    for filename in os.listdir(image_folder):
        image_path = os.path.join(image_folder, filename)

        # Extracting features for the current image
        features = extract_features(image_path)
        if features is None:
            continue  # Skiping if feature extraction failed

        # Computing similarity (Cosine Similarity)
        similarity = np.dot(reference_features, features) / (np.linalg.norm(reference_features) * np.linalg.norm(features))

        # Assigning labels (threshold default =0.8)
        label = "1" if similarity >= threshold else "0"

        # Renaming and saving the image
        new_filename = f"{label}_{filename}"
        new_path = os.path.join(output_folder, new_filename)
        cv2.imwrite(new_path, cv2.imread(image_path))

        print(f"Processed: {filename} → {new_filename} (Similarity: {similarity:.2f})")

In [ ]:
image_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/splitted_images"
feature_file = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/grayscale_feature/extracted_grayscale_feature.csv"
output_folder = "/content/gdrive/MyDrive/Arecanut_Detection_with_ML/Classification/Demo_Classification/grayscale_feature/labelled_images"

# Classify images
classify_and_rename_images(image_folder, feature_file, output_folder)

Processed: tile_0.png → 0_tile_0.png (Similarity: 0.05)
Processed: tile_2.png → 1_tile_2.png (Similarity: 0.96)
Processed: tile_1.png → 0_tile_1.png (Similarity: 0.18)
Processed: tile_3.png → 1_tile_3.png (Similarity: 0.95)
Processed: tile_4.png → 1_tile_4.png (Similarity: 0.99)
Processed: tile_5.png → 1_tile_5.png (Similarity: 1.00)
Processed: tile_6.png → 1_tile_6.png (Similarity: 1.00)
Processed: tile_9.png → 1_tile_9.png (Similarity: 0.88)
Processed: tile_8.png → 0_tile_8.png (Similarity: 0.13)
Processed: tile_7.png → 0_tile_7.png (Similarity: 0.30)
Processed: tile_10.png → 1_tile_10.png (Similarity: 0.84)
Processed: tile_11.png → 1_tile_11.png (Similarity: 0.97)
Processed: tile_12.png → 1_tile_12.png (Similarity: 0.99)
Processed: tile_13.png → 1_tile_13.png (Similarity: 0.99)
Processed: tile_14.png → 1_tile_14.png (Similarity: 0.81)
Processed: tile_15.png → 0_tile_15.png (Similarity: 0.38)
Processed: tile_17.png → 1_tile_17.png (Similarity: 0.88)
Processed: tile_16.png → 1_tile_16